In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from shapely.geometry import Point
import datetime as dt

In [6]:
directory = "Data/Raw/tree_census_2015.csv"
dff = pd.read_csv(directory)

df = dff[['health','borough','latitude', 'longitude']]
df.sample(10)


,health,borough,latitude,longitude
26565,Good,Brooklyn,40.689926,-73.968089
527994,Good,Brooklyn,40.691311,-73.989218
54709,Good,Queens,40.764003,-73.816672
58971,Good,Manhattan,40.732645,-74.004652
80680,Good,Bronx,40.835039,-73.823196
312328,Good,Staten Island,40.539619,-74.180296
328692,Good,Staten Island,40.630483,-74.108085
638730,Good,Queens,40.709227,-73.910011
651087,Good,Brooklyn,40.682950,-73.985065
298676,Good,Staten Island,40.573860,-74.121329


In [7]:
# get census tract to coordinate conversion

# Load the census tract shapefile
tracts = gpd.read_file('Data/Raw/tracts2020_shapefile/nyct2020.shp')
tracts = tracts.to_crs(epsg = 4326)

coords_df = df[["latitude","longitude"]].drop_duplicates().reset_index(drop=True)
coords = [tuple(record) for record in coords_df.to_records(index=False)] #[(40.735324, -73.998004), (40.715595,	-73.987030)]

# Create a GeoDataFrame from the coordinates
geometry = [Point(lon, lat) for lat, lon in coords]
geo_df = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")
geo_df["latitude"] = coords_df["latitude"]
geo_df["longitude"] = coords_df["longitude"]

# Perform a spatial join to match points to census tracts
ct_lookup = gpd.sjoin(geo_df, tracts, how='left', predicate="within")[["latitude", "longitude", "GEOID"]]

# merge our lats/longs with the lookup table we created
df_ct = df.merge(ct_lookup, left_on=["latitude", "longitude"], right_on=["latitude", "longitude"])
df_ct.drop(['latitude','longitude'], axis=1, inplace=True)
df_ct

,health,borough,GEOID
0,Fair,Queens,36081073900
1,Fair,Queens,36081097300
2,Good,Brooklyn,36047044901
3,Good,Brooklyn,36047044902
4,Good,Brooklyn,36047016500
...,...,...,...
683783,Good,Brooklyn,36047051900
683784,Good,Queens,36081070700
683785,Good,Staten Island,36085020100
683786,Good,Bronx,36005023502


In [8]:
# group by tract number to get tree counts

df_health = df_ct.groupby(['GEOID','health'])['borough'].count().rename('count').reset_index()
df_15 = df_health.pivot_table(index='GEOID', columns='health', values='count').reset_index()
df_15['year']='2015'
df_15['num_trees'] = df_15['Fair'].fillna(0) +df_15['Good'].fillna(0) +df_15['Poor'].fillna(0)
df_15

health,GEOID,Fair,Good,Poor,year,num_trees
0,36005000200,28.0,341.0,7.0,2015,376.0
1,36005000400,58.0,361.0,21.0,2015,440.0
2,36005001600,50.0,441.0,12.0,2015,503.0
3,36005001901,24.0,64.0,2.0,2015,90.0
4,36005001902,74.0,171.0,14.0,2015,259.0
...,...,...,...,...,...,...
2302,36085030301,14.0,523.0,16.0,2015,553.0
2303,36085030302,63.0,637.0,30.0,2015,730.0
2304,36085031901,37.0,335.0,12.0,2015,384.0
2305,36085031902,115.0,315.0,49.0,2015,479.0


In [9]:
########### 2005 Trees

directory = "Data/Raw/tree_census_2005.csv"
dff = pd.read_csv(directory)

df = dff[['status','latitude','longitude','boroname']]
df


/var/folders/w0/4r14_1sj0j7fl6nz7v1nwgkr0000gp/T/ipykernel_2789/2797896108.py:4: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  dff = pd.read_csv(directory)


,status,latitude,longitude,boroname
0,Good,40.632653,-74.000245,Brooklyn
1,Good,40.620084,-73.901453,Brooklyn
2,Good,40.617996,-73.899111,Brooklyn
3,Good,40.619694,-73.901003,Brooklyn
4,Good,40.618323,-73.899467,Brooklyn
...,...,...,...,...
592367,Good,40.586260,-74.148797,5
592368,Good,40.586090,-74.149013,5
592369,Good,40.585802,-74.149156,5
592370,Good,40.585802,-74.149156,5


In [10]:
# get census tract to coordinate conversion

# Load the census tract shapefile
tracts = gpd.read_file('Data/Raw/tracts2020_shapefile/nyct2020.shp')
tracts = tracts.to_crs(epsg = 4326)

coords_df = df[["latitude","longitude"]].drop_duplicates().reset_index(drop=True)
coords = [tuple(record) for record in coords_df.to_records(index=False)] #[(40.735324, -73.998004), (40.715595,	-73.987030)]

# Create a GeoDataFrame from the coordinates
geometry = [Point(lon, lat) for lat, lon in coords]
geo_df = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")
geo_df["latitude"] = coords_df["latitude"]
geo_df["longitude"] = coords_df["longitude"]

# Perform a spatial join to match points to census tracts
ct_lookup = gpd.sjoin(geo_df, tracts, how='left', predicate="within")[["latitude", "longitude", "GEOID"]]

# merge our lats/longs with the lookup table we created
df_ct = df.merge(ct_lookup, left_on=["latitude", "longitude"], right_on=["latitude", "longitude"])
df_ct.drop(['latitude','longitude'], axis=1, inplace=True)
df_ct

,status,boroname,GEOID
0,Good,Brooklyn,36047021600
1,Good,Brooklyn,36047070601
2,Good,Brooklyn,36047070601
3,Good,Brooklyn,36047070601
4,Good,Brooklyn,36047070601
...,...,...,...
592367,Good,5,36085027704
592368,Good,5,36085027704
592369,Good,5,36085027704
592370,Good,5,36085027704


In [17]:
df_health = df_ct.groupby(['GEOID','status'])['boroname'].count().rename('count').reset_index()
df_05 = df_health.pivot_table(index='GEOID',columns='status',values='count').reset_index()
df_05['year']='2005'
df_05['num_trees'] = df_05['Excellent'].fillna(0)+ df_05['Good'].fillna(0)+ df_05['Poor'].fillna(0)
df_05


status,GEOID,Dead,Excellent,Good,Poor,year,num_trees
0,36005000200,3.0,92.0,135.0,13.0,2005,240.0
1,36005000400,8.0,42.0,149.0,5.0,2005,196.0
2,36005001600,3.0,78.0,134.0,19.0,2005,231.0
3,36005001901,NaN,33.0,10.0,3.0,2005,46.0
4,36005001902,1.0,29.0,39.0,5.0,2005,73.0
...,...,...,...,...,...,...,...
2283,36085030301,10.0,45.0,357.0,50.0,2005,452.0
2284,36085030302,16.0,155.0,515.0,41.0,2005,711.0
2285,36085031901,11.0,7.0,192.0,11.0,2005,210.0
2286,36085031902,15.0,7.0,221.0,20.0,2005,248.0


### Impute missing values for either 05 or 15

In [51]:
# calculate the avg percent change between 05 and 15
combined = df_05[['GEOID', 'year', 'num_trees']].merge(df_15[['GEOID', 'year', 'num_trees']], on='GEOID')
combined['pct_chng'] = ((combined['num_trees_y'] - combined['num_trees_x']) / combined['num_trees_x'])
avg_pct_chng = combined['pct_chng'].mean()  

# find 2015 rows not in 2005 df
missing_05 = df_15[['GEOID','year','num_trees']][df_15['GEOID'].isin(df_05['GEOID'])==False] 
missing_05['num_trees'] = round(missing_05['num_trees'] * avg_pct_chng)
missing_05['year'] = '2005'

# find 2005 rows not in 2015 df
missing_15 = df_05[['GEOID','year','num_trees']][df_05['GEOID'].isin(df_15['GEOID'])==False] # find 2015 rows not in 2005 df
missing_15['num_trees'] = round(missing_15['num_trees'] * (1+avg_pct_chng))
missing_15['year'] = '2015'
missing_15

### Combine tree counts by year
stacked = pd.concat([df_05[['GEOID', 'year','num_trees']], missing_05, missing_15, df_15[['GEOID', 'year', 'num_trees']]])
stacked


,GEOID,year,num_trees
0,36005000200,2005,240.0
1,36005000400,2005,196.0
2,36005001600,2005,231.0
3,36005001901,2005,46.0
4,36005001902,2005,73.0
...,...,...,...
2302,36085030301,2015,553.0
2303,36085030302,2015,730.0
2304,36085031901,2015,384.0
2305,36085031902,2015,479.0


In [53]:
## Change frequency of counts to yearly

stacked['year'] = pd.to_datetime(stacked['year'])
yearly = stacked.groupby('GEOID')[['year','num_trees']].apply(lambda x: x.resample("YE", on='year').mean()).apply(lambda x: x.interpolate(method='linear')).reset_index()
                    
# add more years after 2015, and limit start to 2010
full_date_range = pd.date_range(start='2010-12-31', end='2023-12-31', freq='YE')
all_years = yearly.groupby('GEOID')[['year','num_trees']].apply(lambda x: x.set_index('year').reindex(full_date_range)).reset_index()
all_years

,GEOID,level_1,num_trees
0,36005000200,2010-12-31,308.0
1,36005000200,2011-12-31,321.6
2,36005000200,2012-12-31,335.2
3,36005000200,2013-12-31,348.8
4,36005000200,2014-12-31,362.4
...,...,...,...
32349,36085032300,2019-12-31,NaN
32350,36085032300,2020-12-31,NaN
32351,36085032300,2021-12-31,NaN
32352,36085032300,2022-12-31,NaN


### Extrapolate values for 2016-2023

In [54]:
kw = dict(method="quadratic", fill_value="extrapolate")
extrap = all_years.groupby('GEOID')['num_trees'].apply(lambda x: x.interpolate(**kw)).reset_index()
extrap = pd.concat([extrap.drop('level_1',axis=1), all_years['level_1']],axis=1)  # get year column back

extrap.rename(columns={'level_1':'year'}, inplace=True)
extrap['year'] = extrap['year'].dt.year
extrap

,GEOID,num_trees,year
0,36005000200,308.0,2010
1,36005000200,321.6,2011
2,36005000200,335.2,2012
3,36005000200,348.8,2013
4,36005000200,362.4,2014
...,...,...,...
32349,36085032300,102.4,2019
32350,36085032300,100.5,2020
32351,36085032300,98.6,2021
32352,36085032300,96.7,2022


In [56]:
# change negative counts created through extrapolation to zero
extrap.loc[extrap.num_trees<0, 'num_trees']=0

# change dtype to int
extrap.num_trees = extrap.num_trees.astype(int)

In [58]:
# save to cleaned data
# extrap.to_parquet("Data/Cleaned/tree_census.parquet")